# Overview

The goal of this file is to load the raw database files that ia in the `raw_data` folder, extract and format the data in a way that is usefull for training the model. The output files are saved in the `datasets` folder.

Produces:
- 'datasets/train.csv' : training data. 80% of the data.
- 'datasets/test.csv' : testing data. 20% of the data.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
from tqdm import tqdm
import sys
import os

In [ ]:
#load data from SQL server
server = 'localhost'
database_names = ['Bucket_110914_1', 'Bucket_110914_2'] #Bucket_<ID>_<#>
username = 'SA'
password = input('Enter password: ')
port = '1433'
driver = 'ODBC+Driver+17+for+SQL+Server'

data = {}
for database in tqdm(database_names):
    engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server}:{port}/{database}?driver={driver}')
    
    inspector = inspect(engine)
    table_names = inspector.get_table_names()

    # Create a dictionary of dataframes
    dfs = {}

    # Loop through table names and for each table, execute a SQL query and load the result into a pandas DataFrame
    for table in tqdm(table_names):
        query = f'SELECT * FROM {table}'
        dfs[table] = pd.read_sql_query(query, engine)
    data = {**data, **dfs}

#load data from spreadsheet
data = {**data, **pd.read_excel('raw_data/Bucket_110915.xlsx', sheet_name=None), **pd.read_excel('raw_data/Bucket_11100.xlsx', sheet_name=None), 'CONSTRUCTION_HIST': pd.read_csv('raw_data/Bucket_11101.csv')}


In [ ]:
#write all data to parquet files
data = {'CONSTRUCTION_HIST': pd.read_csv('raw_data/Bucket_111001.csv')}
for table in tqdm(data):
    data[table].to_parquet(f'./datasets/{table}.parquet')

In [ ]:
#load and write the IRI data csv file to overwrite the MON_HSS_PROFILE_SECTION parquet file
iri_data = pd.read_csv('raw_data/IRI_DATA.csv')
iri_data.to_parquet('./datasets/MON_HSS_PROFILE_SECTION.parquet')

In [ ]:
#load all data from parquet files
data_dir = './datasets/'
data = {}
for file in tqdm(os.listdir(data_dir)):
    if file.endswith('.parquet'):
        data[".".join(file.split('.')[:-1])] = pd.read_parquet(data_dir + file)

# Preprocessing

This will happen in a few steps.

- Weather data will be loaded and processed into a single useful dataframe.
- IRI dataframe will be loaded and columns from other dataframes will be added to it.
- The final result will be saved to another parquet file.

In [ ]:
# Data to use
# {'MON_HSS_PROFILE_SECTION.parquet':
#   Index(['VISIT_DATE', 'STATE_CODE', 'SHRP_ID', 'MRI'], dtype='object'),
# 'TRF_HIST_EST_ESAL.parquet':
#  Index(['STATE_CODE', 'SHRP_ID', 'YEAR_HIST_EST', 'AADT_ALL_VEHIC'], dtype='object'),
# 'MON_DIS_AC_CRACK_INDEX.parquet':
#  Index(['STATE_CODE', 'SHRP_ID', 'SURVEY_DATE', 'MEPDG_TRANS_CRACK_LENGTH_AC'], dtype='object'),
# 'CONSTRUCTION_HIST.parquet': Index(['STATE_CODE', 'SHRP_ID', 'IMP_DATE', 'IMP_TYPE'], dtype='object')}

# The goal is to merge them all together with SHRP_ID and STATE_CODE as the primary keys

FINAL_DATA_DIR = './training_data/'
IRI = data['MON_HSS_PROFILE_SECTION'].set_index(['SHRP_ID', 'STATE_CODE'])
IRI = IRI[['VISIT_DATE', 'IRI_LEFT_WHEEL_PATH', 'IRI_RIGHT_WHEEL_PATH']]
IRI.reset_index(inplace=True)
IRI['VISIT_DATE'] = pd.to_datetime(IRI['VISIT_DATE'], format='%m/%d/%Y')
IRI['IRI_LEFT_WHEEL_PATH'] = IRI['IRI_LEFT_WHEEL_PATH'].astype(float)
IRI['IRI_RIGHT_WHEEL_PATH'] = IRI['IRI_RIGHT_WHEEL_PATH'].astype(float)
IRI = IRI.groupby(['SHRP_ID', 'STATE_CODE', 'VISIT_DATE'], as_index=False).agg({
    'IRI_LEFT_WHEEL_PATH': 'mean',
    'IRI_RIGHT_WHEEL_PATH': 'mean'
})
IRI.set_index(['SHRP_ID', 'STATE_CODE'], inplace=True)

IRI.to_parquet(FINAL_DATA_DIR + 'IRI-only.parquet')
IRI.to_csv(FINAL_DATA_DIR + 'IRI-only.csv')

ESAL = data['TRF_HIST_EST_ESAL'].set_index(['SHRP_ID', 'STATE_CODE'])
ESAL = ESAL['AADT_ALL_VEHIC']

CRACK = data['MON_DIS_AC_CRACK_INDEX'].set_index(['SHRP_ID', 'STATE_CODE'])
CRACK = CRACK['MEPDG_TRANS_CRACK_LENGTH_AC']
CRACK = CRACK.astype(float)

CONSTRUCTION = data['CONSTRUCTION_HIST'].set_index(['SHRP_ID', 'STATE_CODE'])
CONSTRUCTION = CONSTRUCTION[['IMP_DATE', 'IMP_TYPE']]

# Merge traffic and IRI
final = IRI.merge(ESAL, how='left', left_index=True, right_index=True)
# Merge crack data
final = final.merge(CRACK, how='left', left_index=True, right_index=True)

final = final.reset_index()


final = final.fillna(-1)
final = final.groupby(['SHRP_ID', 'STATE_CODE', 'VISIT_DATE', 'IRI_LEFT_WHEEL_PATH', 'IRI_RIGHT_WHEEL_PATH'], as_index=False).agg({
    'AADT_ALL_VEHIC': 'mean',
    'MEPDG_TRANS_CRACK_LENGTH_AC': 'mean'
})

final.set_index(['SHRP_ID', 'STATE_CODE'], inplace=True)
# remove duplicates and de-NAN the values
final = final[~final.duplicated(keep='first')]
# replace MEPDG_TRANS_CRACK_LENGTH_AC 0s with -1s
final['MEPDG_TRANS_CRACK_LENGTH_AC'].replace(0, -1, inplace=True)

# save to parquet and csv
final.to_parquet(FINAL_DATA_DIR + 'final_data.parquet')
final.to_csv(FINAL_DATA_DIR + 'final_data.csv')

# save construction data to parquet and csv
CONSTRUCTION.to_parquet(FINAL_DATA_DIR + 'construction_data.parquet')
CONSTRUCTION.to_csv(FINAL_DATA_DIR + 'construction_data.csv')